<img src="https://www.th-koeln.de/img/logo.svg" style="float:right;" width="200">

# 12th exercise: <font color="#C70039">First Reinforcement Learning Game (*Frozen Lake*) using OpenAI Gym</font>
* Course: AML
* Lecturer: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Author of notebook: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>. This notebook is based on the great post and notebook from [Rodolfo Mendes](https://morioh.com/p/18a96b9091d3).
* Date:   10.01.2023
* Student: Ali Ünal

<img src="https://launchyourintelligentapphome.files.wordpress.com/2019/05/frozenlake_legended.png?w=531" style="float: center;" width="600">

---------------------------------
**GENERAL NOTE 1**: 
Please make sure you are reading the entire notebook, since it contains a lot of information on your tasks (e.g. regarding the set of certain paramaters or a specific computational trick), and the written mark downs as well as comments contain a lot of information on how things work together as a whole. 

**GENERAL NOTE 2**: 
* Please, when commenting source code, just use English language only. 
* When describing an observation please use English language, too.
* This applies to all exercises throughout this course.

---------------------------------

### <font color="ce33ff">DESCRIPTION</font>:

#### OpenAI Gym
In this exercise you will be using Python and OpenAI Gym to develop your reinforcement learning algorithm. The Gym library is a collection of environments that can be used freely with the reinforcement learning algorithms.

Gym has a ton of environments ranging from simple text based games to Atari games like Breakout and Space Invaders. The library is intuitive to use and simple to install. Just run **pip install gym** and you are good to go! The link to Gym's installation instructions, requirements, and documentation is included in the description. 

Further reading about OpenAI Gym is available under https://www.gymlibrary.dev/.
This notebook is based on this great post and notebook from [Rodolfo Mendes](https://morioh.com/p/18a96b9091d3).

#### Frozen Lake
This description of the game is copied directly from Gym's website. 

*Winter is coming. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water and die (Game over). At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend. The surface is described using a grid like the following:*

* SFFF
* FHFH
* FFFH
* HFFG

This grid is your environment! S is your (the agent's) starting point and it's safe. F represents the frozen surface and is also safe. H represents a hole and if your agent steps in a hole in the middle of a frozen lake, the game is over because the agent dies. Finally, G represents the goal, which is the space on the grid where the frisbee is located.

The agent can navigate *left, right, up, down* and the episode ends when the agent reaches the goal or falls in a hole. It receives a reward of **1** if it reaches the goal and **0** otherwise.

Here is the summary:
<img src="./images/FrozenLake.States.Rewards.png" style="float: center;" width="800">

---------------------------------

### <font color="FFC300">TASKS</font>:
The tasks that you need to work on within this notebook are always indicated below as bullet points. 
If a task is more challenging and consists of several steps, this is indicated as well. 
Make sure you have worked down the task list and commented your doings. 
This should be done by using markdown.<br> 
<font color=red>Make sure you don't forget to specify your name and your matriculation number in the notebook.</font>

**YOUR TASKS in this exercise are as follows**:
1. import the notebook to Google Colab or use your local machine.
2. make sure you specified you name and your matriculation number in the header below my name and date. 
    * set the date too and remove mine.
3. read the entire notebook carefully 
    * add comments whereever you feel it necessary for better understanding
    * run the notebook for the first time. 
4. install gym into your env!
5. You will train an agent to play the *Frozen Lake* game using Q-learning and you will get a playback of how the agent does after being trained.
6. Again the task: Your agent has to navigate the grid by staying on the frozen surface without falling into any holes until it reaches the frisbee. If it reaches the frisbee, it wins with a reward of plus one. If it falls in a hole, it loses and receives no points for the entire episode.
7. Your tasks are highlighted in the notebook (see below)
---------------------------------

### Imports 
import all important libs including gym

In [5]:
import numpy as np
import gym
import random
import time
from   IPython.display import clear_output

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


### Creating the Environment
For creating your environment, just call *gym.make()* and pass a string of the name of the environment you want to set up. 
All the environments with their corresponding names you can use here are available on Gym's website (see above).
With this *env* object, you are able to query for information about the environment, sample states and actions, retrieve rewards and have your agent navigate the frozen lake. That is all made available to you conveniently with Gym.

In [6]:
env = gym.make("FrozenLake-v1")

### Creating the Q-Table
Now, construct your Q-table, and initialize all the Q-values to zero for each state-action pair.
The number of rows in the table is equivalent to the size of the state space in the environment, and the number of columns is equivalent to the size of the action space (see above). You can get this information using *env.observation_space.n* and *env.action_space.n* as shown below in the code. Then, you can use this information to build the Q-table and initialize it with zeros.

In [3]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [4]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### Initializing Q-Learning hyperparameters
Now, we're going to create and initialize all the parameters needed to implement the Q-learning algorithm.

First, with *num_episodes*, you define the total number of episodes you want the agent to play during training. Then, with *max_steps_per_episode*, you define a maximum number of steps that your agent is allowed to take within a single episode. So, if by the 100th step, the agent has not reached the frisbee or fallen through a hole, then the episode will terminate with the agent receiving zero points.

Next, you will set your *learning_rate* and your *discount_rate* as well, which was represented with the symbol (lambda) in the course slides (keyword: discounted return G_t).

Now, the last four parameters are all related to the exploration-exploitation dilemma with respect to the epsilon-greedy policy. You are initializing your *exploration_rate* to **1** and setting the *max_exploration_rate* to **1** and a *min_exploration_rate* to **0.01**. The *max* and *min* are just bounds to how large or small your exploration rate can be. Remember, the exploration rate was represented with the symbol (epsilon) when discussed in the course slides.

Lastly, you will set the *exploration_decay_rate* to **0.01** to determine the rate at which the *exploration_rate* will decay.

**YOUR <font color="FFC300">TASK</font> in this exercise is as follows** (point 7 from the task list above):

All of the above parameters can change!
Your task is to create a *testplan* and tune all parameters by yourself and observe how they influence and change the performance of the algorithm. 
Make notes! They will help you during the exam.

In [5]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 100
max_exploration_rate = 100
min_exploration_rate = 0.01
exploration_decay_rate = 0.01

Create a list to hold all of the rewards you will get from each episode. 
By means of this you can observe how your game score changes over time.

In [6]:
rewards_all_episodes = []

In the following code section, the entire Q-learning algorithm is implemented as discussed in detail in the AML course. 
When this code is executed, this is exactly where the training will take place. 
* The first for-loop contains everything that happens within a single episode. 
* The second nested loop contains everything that happens for a single time-step.

Read all the red comments, as they contain lots of important information on the implementation.

In [7]:
# Q-learning algorithm

# loop: for a single episode
for episode in range(num_episodes):
    # initialize 'new episode' parameters
    state = env.reset()
    ''' The done variable just keeps track of whether or not your episode is finished.
    Initialize it to False when first starting the episode and you will see later where 
    it will get updated to notify you when the episode is over.'''
    done = False
    
    ''' Keep track of the rewards within the current episode as well.
    Hence, set rewards_current_episode = 0 since you start 
    with no rewards at the beginning of each episode.'''
    rewards_current_episode = 0

    # nested loop: for a single time-step
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        '''For each time-step within an episode set your exploration_rate_threshold 
        to a random number between 0 and 1. This will be used to determine whether 
        your agent will explore or exploit the environment in this time-step.'''
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()

        # Take new action
        '''After action is chosen, take that action by calling step() on your env object and 
        pass your action to it. The function step() returns a tuple containing the new state, 
        the reward for the action you took, whether or not the action ended the episode and 
        diagnostic information regarding the environment (helpful for debugging).'''
        new_state, reward, done, info = env.step(action)

        # Update Q-table for Q(s,a)
        '''Compare this implementation with the equation in the course slides.'''
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
        learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        '''Set your current state to the new_state that was returned when taking the last action 
        and then update the rewards from your current episode by adding the reward you received 
        for your previous action.'''
        # Set new state
        state = new_state
        # Add new reward 
        rewards_current_episode += reward 
        '''Then, check to see if your last action ended the episode 
        (game over by agent stepping in a hole or reaching the goal)! 
        If the action did end the episode, then jump out of this loop and start the next episode.
        Otherwise, transition to the next time-step.'''
        if done == True: 
            break
            

    # Exploration rate decay
    '''Once an episode is finished, you need to update your exploration_rate using exponential decay, 
    which just means that the exploration rate decays at a rate proportional to its current value. 
    You can decay the exploration_rate using the formula above, which makes use of all the exploration 
    rate parameters that were defined above in the hyperparameter section.'''
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    # Add current episode reward to total rewards list and move on to the next episode
    rewards_all_episodes.append(rewards_current_episode)


### All episodes training completed
After all episodes are finished you now just calculate the average reward per thousand episodes from your list that contains the rewards for all episodes so that you can print it out and see how the rewards changed over time.

In [8]:
# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/10)
count = 10

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/10)))
    count += 10

********Average reward per thousand episodes********

10 :  0.0
20 :  0.0
30 :  0.0
40 :  0.1
50 :  0.0
60 :  0.0
70 :  0.0
80 :  0.0
90 :  0.0
100 :  0.0
110 :  0.0
120 :  0.0
130 :  0.0
140 :  0.1
150 :  0.0
160 :  0.1
170 :  0.0
180 :  0.0
190 :  0.0
200 :  0.0
210 :  0.0
220 :  0.0
230 :  0.0
240 :  0.1
250 :  0.0
260 :  0.1
270 :  0.0
280 :  0.0
290 :  0.0
300 :  0.1
310 :  0.0
320 :  0.0
330 :  0.0
340 :  0.0
350 :  0.1
360 :  0.0
370 :  0.0
380 :  0.0
390 :  0.0
400 :  0.0
410 :  0.0
420 :  0.0
430 :  0.1
440 :  0.0
450 :  0.0
460 :  0.0
470 :  0.0
480 :  0.0
490 :  0.0
500 :  0.1
510 :  0.0
520 :  0.0
530 :  0.1
540 :  0.0
550 :  0.0
560 :  0.0
570 :  0.1
580 :  0.2
590 :  0.0
600 :  0.1
610 :  0.0
620 :  0.0
630 :  0.30000000000000004
640 :  0.30000000000000004
650 :  0.1
660 :  0.1
670 :  0.0
680 :  0.4
690 :  0.30000000000000004
700 :  0.2
710 :  0.2
720 :  0.30000000000000004
730 :  0.1
740 :  0.30000000000000004
750 :  0.4
760 :  0.4
770 :  0.1
780 :  0.5
790 :  0.1
800 : 

### Interpretation

From this print, you can see that the average reward per thousand episodes did indeed progress over time. When the algorithm first started training, the first thousand episodes only averaged a reward of almost **0.18**, but by the time it got to its last thousand episodes, the reward drastically improved to almost **0.7**.

Let's take a second to understand how you can interpret these results. Your agent played **10000** episodes. At each time step within an episode, the agent received a reward of **1** if it reached the frisbee, otherwise, it received a reward of **0**. If the agent did indeed reach the frisbee, then the episode finished at that time-step.

Hence, that means for each episode, the total reward received by the agent for the entire episode is either **1** or **0**. So, for the first thousand episodes, you can interpret this score as meaning that **18%** of the time the agent received a reward of **1** and won the episode. And by the last thousand episodes from a total of **10000**, the agent was winning almost **70%** of the time.

By analyzing the grid of the game, you can see it is a lot more likely that the agent would fall in a hole or perhaps reach the max time steps than it is to reach the frisbee, so reaching the frisbee **70%** of the time is not too bad, especially since the agent had no explicit instructions to reach the frisbee. It learned that this is the correct thing to do.

* SFFF
* FHFH
* FFFH
* HFFG

At last, print out your updated Q-table to see how that has transitioned from its initial state of all zeros.

In [9]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)



********Q-table********

[[0.56440386 0.51208794 0.49554071 0.50790574]
 [0.22323527 0.28106842 0.23187829 0.49163638]
 [0.43252568 0.23723976 0.14255076 0.22340176]
 [0.13042745 0.00788836 0.00345618 0.0041471 ]
 [0.59543381 0.35354491 0.24749726 0.44034705]
 [0.         0.         0.         0.        ]
 [0.44750198 0.12342678 0.1635377  0.08002332]
 [0.         0.         0.         0.        ]
 [0.45479025 0.45882903 0.43527271 0.64272999]
 [0.46476269 0.69590813 0.48528143 0.42525724]
 [0.68408806 0.38594584 0.35639477 0.37159112]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.47625507 0.36759817 0.78664713 0.43248723]
 [0.73831219 0.88141163 0.7381083  0.73261588]
 [0.         0.         0.         0.        ]]


| num_episodes | max_steps_per_episode | learning_rate | discount_rate | exploration_rate | max_exploration_rate | min_exploration_rate |  | Average reward per thousand episodes | Last episode reward |
|-------------| --- | --- |
| 10000 | 100 | 0.1 | 0.99 | 1 | 1 | 0.01 | 0.01 | 0.145 | 0.69 |
| 10000 | 100 | 0.2 | 0.99 | 1 | 1 | 0.01 | 0.01 | 0.47 | 0.976 |
| 10000 | 100 | 0.1 | 0.8 | 1 | 1 | 0.01 | 0.01 | 0.176 | 0.359 |
| 10000 | 200 | 0.1 | 0.99 | 1 | 1 | 0.01 | 0.01 | 0.493 | 0.709 |
| 10000 | 400 | 0.1 | 0.99 | 1 | 1 | 0.01 | 0.01 | 0.178 | 0.719 |
| 20000 | 100 | 0.1 | 0.99 | 1 | 1 | 0.01 | 0.01 | 0.21 | 0.687 |


num_episode = 1000
max_steps_per_episode = 100000000
learning rate = 0.1
discount_rate = 0.99
exploration_rate = 10
max_exploration_rate = 10
min_exploration_rate = 0.001
exploration_decay_rate = 0.01

Testing of min_exploration_rate 
0.01 ca 60 %
0.001 ca 65%
0.0001 ca 30-35%
0.00001 ca 40%
0.000001 ca 60%
0.0000001 ca 30%

Testing of exploration and max exploration rate
1 ca 19%
10 ca 68%
100 ca 69%
1000 ca 68%
10000 ca 68%
doesnt seem to get better
100000 ca 66% nothing changes

Testint learning rate

num_episode = 1000
max_steps_per_episode = 100000000
learning rate = 0.1
discount_rate = 0.99
exploration_rate = 100
max_exploration_rate = 100
min_exploration_rate = 0.001
exploration_decay_rate = 0.01

0.1 ca 67%
0.2 ca 67%
0.3 ca 68%
0.4 ca 65 %
0.5 ca 65%
0.6 ca 60% can also be 65%
0.9 ca 40%
0.8 ca 50%
0.7 ca 56%
1.0 ca 0% bis 1.2%
1.5 ca 3.1 %


Discount Rate Testing:

num_episode = 1000
max_steps_per_episode = 100000000
learning rate = 0.3
discount_rate = 0.99
exploration_rate = 100
max_exploration_rate = 100
min_exploration_rate = 0.01
exploration_decay_rate = 0.01


0.99 ca  66%
0.89 ca 38%
0.79 ca 32%
0.69 ca 34%
0.59 ca 18%
0.29 ca 8%!


Testing max_steps_per_episode
num_episode = 1000
max_steps_per_episode = 1000
learning rate = 0.3
discount_rate = 0.99
exploration_rate = 100
max_exploration_rate = 100
min_exploration_rate = 0.01
exploration_decay_rate = 0.01

1000 ca 73%
100 ca 79% at the end


